In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import mannwhitneyu
from sklearn.metrics import silhouette_score

product = "Geschirrspüler"

df = pd.read_csv(f"{product}/{product}_results_per_model.csv", sep=",", dtype={"Vendor": str})
#df = pd.read_csv(f"{product}/{product}_first_timepoint_results.csv", sep=",", dtype={"Vendor": str}) # uncomment if only first timepoint should be analyzed

df_original = df.copy()
#df = df[(df['Manufacturer'] != "Miele") & (df['Manufacturer'] != "Liebherr")]

display(df)

In [ ]:
# Exclude num_price_changes for clustering. Reasons: feature has high specificity but low sensitivity. Also, data spanning a short time span could make this feature unreliable
features = df[["mean_coef_var", "same_price_pct", "cheapest_same_price_pct", "entropy"]]

#features = df[["mean_coef_var"]] # uncomment if only the coef var should be used for clustering

print(features.mean())

# Standardizing the features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features)

**DBSCAN**

In [ ]:
# Define the ranges for eps and min_samples to find the optimal parameters according to the Silhouette Score
eps_range = np.arange(0.1, 1.7, 0.1) # Zwischen 0.1 und 5 getestet: KS: 1, WS: 1.2, GefrierS: 1.1, GeschirrS: 0.7, Lautsprecher: 1.4. Aber: ohne Miele & Liebherr 0.1 optimal bei KS
min_samples_range = range(3, len(df)//2, 2) # min_samples cannot be greater than half of samples. Start with 3, since every manufacturer in the remaining data has at least 3 models

best_eps = None
best_min_samples = None
best_score = -1

# Find the best eps and min_samples
for eps in eps_range:
    for min_samples in min_samples_range:
        db = DBSCAN(eps=eps, min_samples=min_samples)
        labels = db.fit_predict(scaled_data)
        
        # Calculate silhouette score if there is more than one cluster
        if len(set(labels)) > 1:
            score = silhouette_score(scaled_data, labels)
            if score > best_score:
                best_eps = eps
                best_min_samples = min_samples
                best_score = score

print(f"Best eps: {best_eps}")
print(f"Best min_samples: {best_min_samples}")
print(f"Best Silhouette Score: {best_score}")

# Fit DBSCAN with best eps and min_samples
db = DBSCAN(eps=best_eps, min_samples=best_min_samples)
labels_dbscan = db.fit_predict(scaled_data)

# Add cluster labels to the dataframe
df['cluster_dbscan'] = labels_dbscan

# display(df[df['Manufacturer'] == 'Siemens'])
# display(df[df['Manufacturer'] == 'Miele'])

display(df)

# Calculate Silhouette Score and print it
dbscan_silhouette = silhouette_score(scaled_data, df['cluster_dbscan'])
print(f"Silhouette Score for DBSCAN: {dbscan_silhouette}")

**Validating results**

In [ ]:
# Get number of detected clusters
cluster_labels = df['cluster_dbscan'].unique()
print(f"Number of clusters: {len(cluster_labels)}")

# If only one cluster is detected or more than 4 clusters are detected, there most likely is no RPM
if len(cluster_labels) == 1 or len(cluster_labels) > 9:
    print("No RPM has been detected")
else:
    # Create dict with cluster as key and 25th percentile coef_var as value. Reason: If the outlier cluster has at least 25% low outliers, we correctly identify it
    cluster_coefvar = {}
    for cluster in cluster_labels:
        cluster_coefvar[cluster] = df[df['cluster_dbscan'] == cluster]['mean_coef_var'].quantile(0.25)
    
    print(cluster_coefvar)
    print("-------------------------------------------------------------")
    
    # Get the clusters with the lowest and second lowest coef_var
    sorted_clusters = sorted(cluster_coefvar.items(), key=lambda x: x[1])
    lowest = sorted_clusters[0][0]
    second_lowest = sorted_clusters[1][0]

    # Make lowest cluster the suspicious cluster and second lowest the normal cluster
    outliers = df[df['cluster_dbscan'] == lowest]
    non_sus = df[df['cluster_dbscan'] == second_lowest]

    # Define fixed threshold as safety measure, since otherwise low outliers, which are only suspicious compared to the other products, are flagged
    non_sus_same_price_cutoff = 0.65
    sus_cheapest_same_price_cutoff = 0.85

    # Just in case: Only keep low outliers in, remove high outliers from suspicious cluster, if there are any (normally not the case)
    print(f"These were removed from the suspicious cluster via the Same price cutoff ({non_sus_same_price_cutoff})")
    sus_only = outliers[outliers['same_price_pct'] >= non_sus_same_price_cutoff]
    display(outliers[outliers['same_price_pct'] < non_sus_same_price_cutoff])

    # Just in case: Add definitely suspicious models, which were somehow not part of the suspicious cluster
    df_cheap_threshold = df[df['cheapest_same_price_pct'] > 0.85]
    sus_only = pd.concat([sus_only, df_cheap_threshold])
    sus_only = sus_only.drop_duplicates()

    #display(sus_only.sort_values(by='mean_coef_var', ascending=False))

    alpha_level = 0.05

    # Initialize confidence variable, which gets higher if more tests are passed
    confidence = 0

    # Coef_var Mann-Whitney-U test: Mann-Whitney-U test because we have non-normal data and unequal variances
    print(f"Average coef var in sus cluster: {sus_only['mean_coef_var'].mean()}")
    print(f"Average coef var in normal cluster: {non_sus['mean_coef_var'].mean()}")

    sus_cluster_coefvar = sus_only['mean_coef_var']
    normal_cluster_coefvar = non_sus['mean_coef_var']

    t_coefvar, p_coefvar = mannwhitneyu(sus_cluster_coefvar, normal_cluster_coefvar, alternative='less') # One sided

    print(f"P-value for coef-var: {round(p_coefvar, 2)} ({p_coefvar})")

    if p_coefvar < alpha_level:
        rpm_detected = True
        print("Result: One cluster has significantly lower variation in prices than the other(s)")
    else:
        print("Result: Both clusters have similar variation in prices")

    print("-------------------------------------------------------------")
    
    # Num_price_changes Mann-Whitney-U test ---------------------------------------------

    if 'num_price_changes' in df.columns:
        print(f"Average num_price_changes in sus cluster: {sus_only['num_price_changes'].mean()}")
        print(f"Average num_price_changes in normal cluster: {non_sus['num_price_changes'].mean()}")

        if sus_only['num_price_changes'].mean() >= non_sus['num_price_changes'].mean():
            print("Result: The suspicious cluster has actually a higher number of price changes than the normal cluster")
        else:
            sus_cluster_price_changes = sus_only['num_price_changes']
            normal_cluster_price_changes = non_sus['num_price_changes']

            t_changes, p_changes = mannwhitneyu(sus_cluster_price_changes, normal_cluster_price_changes, alternative='less') # One sided

            print(f"P-value for price changes: {round(p_changes, 2)} ({p_changes})")

            if p_changes < alpha_level:
                confidence += 1
                print("Result: The same cluster also has significantly less price_changes than the other(s). Cluster likely represents RPM")
            else:
                print("Result: The same cluster DOES NOT have significantly less price_changes than the other")
            
            #display(sus_only[(sus_only['Manufacturer'] != "Miele") | (sus_only['Manufacturer'] != "Liebherr")].sort_values(by='mean_coef_var', ascending=False))
            #display(non_sus[(non_sus['Manufacturer'] == "Miele") | (non_sus['Manufacturer'] == "Liebherr")].sort_values(by='mean_coef_var', ascending=True))
    else:
        print("Time span too short to validate RPM cluster using num_price_changes")

**Aggregate per Manufacturer**

In [ ]:
if rpm_detected:
    # Get the number of models per manufacturer
    value_counts = df['Manufacturer'].value_counts().reset_index()
    value_counts.columns = ['Manufacturer', 'total_models']
    #display(value_counts)

    # Group by manufacturer and count occurrences
    sus = sus_only.groupby('Manufacturer').size().reset_index(name='sus_count')
    #display(sus)

    # Merge the DataFrames
    result = pd.merge(value_counts, sus, on='Manufacturer', how='left').fillna(0)
    result['sus_count'] = result['sus_count'].astype(int)
    result['pct_suspicious_models'] = round(result['sus_count'] / result['total_models'] * 100, 2)
    result = result.sort_values(by=['pct_suspicious_models', 'sus_count'], ascending=False)

    total_sus_models = result['sus_count'].sum()
    result['pct_of_sus_cluster'] = round(result['sus_count'] / total_sus_models * 100, 2)

    # Only suspicious manufacturers (at least 5 sus models and more than 50 % sus models)
    sus_manufacturers = result[(result['sus_count'] > 4) & (result['pct_suspicious_models'] > 40)]

    if len(sus_manufacturers) == 0:
        print("No suspicious manufacturers found.")
    else:
        print("Suspicious manufacturers (at least 5 suspicious models and more than 40% of all models being suspicious)")
        display(sus_manufacturers)

    print("Full result of analysis")
    result.rename(columns={"total_models": "total_products", "sus_count": "suspicious_products", "pct_suspicious_models": "%_suspicious_products", "pct_of_sus_cluster": "%_of_suspicious_cluster"}, inplace=True)
    display(result)
    # change
    result.to_csv(f"{product}/{product}_sus_manufacturers_coefvar.csv", index=False)
    
    print(f"There are {len(sus_only)} suspicious models, which is {len(sus_only) / len(df) * 100:.2f}% of the total models.")
    if confidence > 0:
        print("High confidence that the marked models are indicating RPM.")
    else:
        print("Medium confidence that the marked models are indicating RPM.")
else:
    print("No RPM has been detected.")

display(sus_only.sort_values(by='entropy', ascending=False))

sus_cluster_label = sus_only['cluster_dbscan'].unique()[0]

In [ ]:
# Merge on both 'Model' and 'mean_coef_var'
df_merged = pd.merge(df_original, sus_only, on=['Model', 'mean_coef_var', 'num_price_changes', 'same_price_pct', 'cheapest_same_price_pct', 'entropy'], how='left', indicator=True)

# Create 'suspicious' column: 'True' if combination exists in smaller df, otherwise 'No'
df_original['suspicious'] = np.where(df_merged['_merge'] == 'both', True, False)

display(df_original)
print(df_original['suspicious'].value_counts()[True])

# Save result to csv
df_original.to_csv(f"{product}/{product}_suspicious.csv", index=False)

**Visualizing clusters using PCA**

In [ ]:
# Perform PCA for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(scaled_data)

print(X_pca)

# Custom legend mapping for the clusters
custom_labels = {}
non_suspicious_counter = 0  # Counter to assign colors to non-suspicious clusters

unique_labels = np.unique(labels_dbscan)

# Define darker, less bright colors for non-suspicious clusters
non_suspicious_colors = ['#4a6fa5', '#468966', '#ad5ead']  # Blue, green, magenta
cluster_colors = {}

for label in unique_labels:
    if label == sus_cluster_label:
        custom_labels[label] = "Suspicious cluster"
        cluster_colors[label] = 'orange'  # Orange for the suspicious cluster
    else:
        custom_labels[label] = f"Non Suspicious cluster {non_suspicious_counter + 1}"
        cluster_colors[label] = non_suspicious_colors[non_suspicious_counter]
        non_suspicious_counter += 1

# Plot the results
plt.figure(figsize=(10, 7))

# Plot each cluster with the assigned color and custom label
for label in unique_labels:
    label_mask = labels_dbscan == label
    plt.scatter(X_pca[label_mask, 0], X_pca[label_mask, 1], 
                label=custom_labels[label], color=cluster_colors[label])

product_eng = {
    "Waschmaschinen": "Washing Machines",
    "Geschirrspüler": "Dishwashers",
    "Gefrierschränke": "Freezers",
    "Kühlschränke": "Refrigerators",
    "Lautsprecher": "Loudspeakers"
}

plt.title(f'DBSCAN Clustering with PCA Reduction: {product_eng[product]}')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.show()